In [1]:
"""
Prepare cav mean and cav std

"""

'\nPrepare cav mean and cav std\n\n'

In [2]:
import xarray as xr
import multimelt.useful_functions as uf
from tqdm.notebook import tqdm
import numpy as np

READ IN DATA

In [3]:
nemo_run0 = 'OPM018' #OPMO16, OPM006, OPM021

nemo_run = nemo_run0

In [4]:
#inputpath_profiles = '/../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'

inputpath_mask = '../../../../../burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
#inputpath_mask = '../../data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

#inputpath_data='../../../../../burgardc/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'


outputpath_profiles_helen = '../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'



In [5]:
# T and S extrapolated to ice draft depth
T_S_2D_isfdraft = xr.open_mfdataset(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS.nc')#.sel(profile_domain=50)

In [6]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [7]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [8]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [9]:
n = 0
for kisf in tqdm(file_isf.Nisf):
    T_isf = uf.choose_isf(T_S_2D_isfdraft['theta_in'],isf_stack_mask, kisf)
    S_isf = uf.choose_isf(T_S_2D_isfdraft['salinity_in'],isf_stack_mask, kisf)

    T_mean_cav = T_isf.mean('mask_coord').to_dataset(name='T_mean')
    S_mean_cav = S_isf.mean('mask_coord').to_dataset(name='S_mean')
    T_std_cav = T_isf.std('mask_coord').to_dataset(name='T_std')
    S_std_cav = S_isf.std('mask_coord').to_dataset(name='S_std')

    
    all_TS_isf = xr.merge([T_mean_cav,S_mean_cav,T_std_cav,S_std_cav])
    all_TS_isf_2D = all_TS_isf.where(T_isf).load()
    
    if n == 0:
        all_TS_all = all_TS_isf_2D.squeeze().drop('Nisf')
    else:
         all_TS_all =  all_TS_all.combine_first(all_TS_isf_2D).squeeze().drop('Nisf')
    n = n+1

    del all_TS_isf_2D

all_TS_all_unstacked = uf.bring_back_to_2D(all_TS_all)
all_TS_all_unstacked.to_netcdf(outputpath_profiles_helen + 'T_S_2D_meanstd_isf_draft_oneFRIS.nc')

  0%|          | 0/35 [00:00<?, ?it/s]

In [10]:
xr.open_dataset(outputpath_profiles_helen + 'T_S_2D_meanstd_isf_draft_oneFRIS.nc')

<xarray.Dataset>
Dimensions:         (y: 726, x: 937, profile_domain: 2, time: 29)
Coordinates:
  * y               (y) float64 -2.132e+06 -2.127e+06 ... 2.233e+06 2.238e+06
  * x               (x) float64 -2.338e+06 -2.333e+06 ... 2.737e+06 2.742e+06
  * time            (time) int64 1980 1981 1982 1983 ... 2005 2006 2007 2008
  * profile_domain  (profile_domain) object 'close_cont_shelf' '1000'
    latitude        (y, x) float64 ...
    longitude       (y, x) float64 ...
Data variables:
    T_mean          (profile_domain, time, y, x) float64 ...
    S_mean          (profile_domain, time, y, x) float64 ...
    T_std           (profile_domain, time, y, x) float64 ...
    S_std           (profile_domain, time, y, x) float64 ...